In [10]:
import os
from pathlib import Path
import anndata as ad
from scipy import sparse
import numpy as np
import scanpy as sc

In [2]:
os.chdir(Path().resolve())
os.getcwd()

'/home/wu/datb1/AutoExtractSingleCell/03.skin_phase2/sample3/raw_data'

In [3]:
# sample_set = set()
# sample_list = [x for x in os.listdir('.') if x.endswith('.gz')]
# for sample in sample_list:
#     sample_set.add('_'.join(sample.split('_')[:3]))
# sample_set = list(sample_set)
# sample_set

[]

In [ ]:
# for sample in sample_set:
#     os.mkdir(sample)
#     for sample_file in sample_set:
#         if sample in sample_file:
#             os.system(f'mv {sample_file} {sample}')
#             if 'barcodes' in sample_file:
#                 os.system(f'mv {os.path.join(sample, sample_file)} {os.path.join(dir_path, sample, "barcodes.tsv.gz")}')
#             elif 'features' in sample_file:
#                 os.system(f'mv {os.path.join(sample, sample_file)} {os.path.join(dir_path, sample, "features.tsv.gz")}')
#             elif 'matrix' in sample_file:
#                 os.system(f'mv {os.path.join(sample, sample_file)} {os.path.join(dir_path, sample, "matrix.mtx.gz")}')

In [5]:
if 'adata_all' in locals():
    del adata_all

sample_set = [x for x in os.listdir('.') if x.startswith('GSM')]

for sample in sample_set:
    adata = sc.read_10x_mtx(sample, var_names='gene_symbols', cache=True)
    sc.pp.filter_cells(adata, min_genes=300)
    print(f"Processing sample {sample}")
    print(adata.shape)
    # drop var col
    # adata.var = pd.DataFrame(index=adata.var_names)
    adata.obs_names = adata.obs_names + '_' + sample
    adata.obs['Batch'] = sample
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    # if adata_all is already defined, concat the new adata to it
    if 'adata_all' in locals():
        adata_all = ad.concat([adata_all, adata], uns_merge='unique')
    else:
        adata_all = adata.copy()
        adata_all.uns['batch'] = []
        adata_all.uns['batch_raw'] = {}

    adata_all.uns['batch'].append(sample)
    adata_all.uns['batch_raw'][sample] = len(adata_all.obs_names)
    
    # if adata_all.obs.shape[0] > 100000:
    #     break # stop if the number of cells is too large

Processing sample GSM7120451_P1_V2
(12970, 36601)
Processing sample GSM7120468_P5_V3
(14752, 36601)
Processing sample GSM7120452_P1_V3
(11768, 36601)
Processing sample GSM7120454_P2_V1
(12673, 36601)
Processing sample GSM7120457_P3_V1
(11310, 36601)
Processing sample GSM7120464_P4_V3
(9290, 36601)
Processing sample GSM7120450_P1_V1
(8347, 36601)
Processing sample GSM7120461_P4_V1
(12390, 36601)
Processing sample GSM7120459_P3_V2
(7598, 36601)
Processing sample GSM7120463_P4_V2
(12468, 36601)
Processing sample GSM7120453_P2_V1
(12020, 36601)
Processing sample GSM7120462_P4_V1
(11248, 36601)
Processing sample GSM7120458_P3_V1
(11539, 36601)
Processing sample GSM7120460_P3_V3
(7848, 36601)
Processing sample GSM7120467_P5_V2
(13762, 36601)
Processing sample GSM7120465_P5_V1
(14041, 36601)
Processing sample GSM7120466_P5_V1
(12407, 36601)
Processing sample GSM7120455_P2_V2
(10529, 36601)
Processing sample GSM7120449_P1_V1
(13554, 36601)
Processing sample GSM7120456_P2_V3
(14995, 36601)


In [9]:
NL_list = ['GSM7120450', 'GSM7120454', 'GSM7120458', 'GSM7120462', 'GSM7120466']
adata_all.obs['Disease'] = 'Psor'
adata_all.obs.loc[adata_all.obs.index.str.contains('|'.join(NL_list)), 'Disease'] = 'Psor Normal'
adata_all.obs.Disease.value_counts()

Disease
Psor           179295
Psor Normal     56214
Name: count, dtype: int64

In [12]:
# downsample to 60000 cells
adata_all = adata_all[np.random.choice(adata_all.obs_names, 60000, replace=False), :].copy()

In [13]:
adata_all.obs.Disease.value_counts()

Disease
Psor           45734
Psor Normal    14266
Name: count, dtype: int64

In [14]:
adata_all.write('sample3_raw.h5ad')